## Problema de CBOW y skip-gram

No tiene en cuenta el **orden de las palabras**, con lo que se puede perder el significado. Además tienen un gran **coste computacional y temporal**.

## Redes neuronales recurrentes (RNN)

Las RNNs procesan **secuencias** y retienen información en una especie de "memoria" (estado). Donde $x_{i}$ y $h_{i-1}$ actúan como input para el siguiente paso, donde se produce $h_{i}$, el output en cada iteración. El output se representa como un vector de números que representa toda la frase en su contexto. De esta manera, se tiene en cuenta el contexto de la frase para producir el siguiente output.

<img src="RNN.png">

## Tipos de RNNs

- **Normal RNN**: Clasificación de imágenes (un input, un output).
- **One-to-many**: Escribir el pie de una imagen (input: imágen, output: pie de imagen con muchas palabras).
- **Many-to-one**: Clasificación de sentimientos (input: oración, output: sentimiento).
- **Many-to-many**: Traducción de textos, clasificación de cada fotograma de un vídeo...

## Problemas de las RNNs

El **gradiente** indica el ajuste a realizar en los pesos con respecto a la variación en el error. Existen dos problemas con los gradientes:

- **Gradientes explosivos**: Gradiente demasiado grande, el algoritmo asigna una importance exageradamente alta a las pesos. Tiene fácil solución truncando los gradientes.
- **Gradientes desaparecidos**: Gradientes demasiado pequeños y el modelo deja de aprender o aprende muy despacio. Sucede a menudo cuando las secuencias son relativamente **largas**. Es más difícil de solucionar.

## Long-short term memory (LSTMs)

Se plantea como una solución para retener en memoria palabras que aparecieron hace mucho tiempo (e.g. $x_{1}$ w.r.t. $x_{6}$) y no perder su información. Mantienen los valores de los gradientes suficientemente altos, realizando un entrenamiento más rápido y de mayor precisión. 

La idea es utilizar un vector a modo de peso ($\epsilon~[0,1]$) que determinará cómo de importante es la información de los pasos anteriores. Los vectores se aplican en un sistema de *puertas*. La puerta de olvido se emprlea para dterminar la importancia del output anterior $h_{i-1}$, la puerta de entrada valora la importancia de un nuevo input $x_{i}$ y la puerta de salida le aplica un peso al output producido $h_{t}$.

Otra alternativa son las llamadas **Gated Recurrent Units (GRUs)**, que tienen un rendimiento similar pero son computacionalmente más eficientes.

El problema de las LSTMs es que los cálculos se hacen en serie, palabra por palabra, lo que ralentiza mucho el entrenamiento. Si la frase es muy larga, sigue siendo un problema.

## Clasificación de texto usando LSTMs con TensorFlow

In [1]:
## Download dataset

!wget https://raw.githubusercontent.com/susanli2016/PyCon-Canada-2019-NLP-Tutorial/master/bbc-text.csv

zsh:1: command not found: wget


In [2]:
## Import necessary libraries
import nltk
nltk.download('stopwords')

import csv
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from nltk.corpus import stopwords
STOPWORDS = set(stopwords.words('english'))

ModuleNotFoundError: No module named 'nltk'

In [3]:
print (tf.__version__)

NameError: name 'tf' is not defined

In [4]:
## Access and visualize the dataset
df = pd.read_csv('bbc-text.csv')
df.head()

NameError: name 'pd' is not defined

In [5]:
df.info()

NameError: name 'df' is not defined

In [6]:
df['category'].value_counts()

NameError: name 'df' is not defined

In [7]:
## Create the dataset

articles = []
labels = []

with open('bbc-text.csv','r') as f:
    reader = csv.reader(f,delimiter=',')
    next(reader) # skip first line
    for row in reader:
        labels.append(row[0])
        article = row[1]
        for word in STOPWORDS:
            token = ' ' + word + ' '
            article = article.replace(token, ' ')
        articles.append(article)
        
print(len(labels))
print(len(articles))

NameError: name 'STOPWORDS' is not defined

In [8]:
# This could have been done directly with pandas
for word in STOPWORDS:
    df['text'] = df['text'].apply(lambda x: x.replace(' '+word+' ',' '))

NameError: name 'STOPWORDS' is not defined

In [9]:
# Neural network parameters
vocab_size = 5000 # This is the maximum number of words to keep
embedding_dim = 64
# Padding parameters (see later for padding definition)
max_length = 200 # Maximum length of all sequences
trunc_type = 'post' #pre or post. Pad (add zeros) either before or after each sequence.
padding_type = 'post' #pre or post. Remove values larger than maximum_length either at the beginning or at the end of each sequence.
oov_tok = '<OOV>' #Used to replace out-of-vocabulary words
training_portion = 0.8

In [10]:
# Train/test split
train_size = int(df.shape[0]*training_portion)

X_train, X_test = df['text'][:train_size], df['text'][train_size:]
y_train, y_test = df['category'][:train_size], df['category'][train_size:]
print (len(X_train), len(y_train))
print (len(X_test), len(y_test))

NameError: name 'df' is not defined

In [11]:
## Create the tokenizer
tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(X_train) # Updates internal vocabulary based on a list of texts
word_index = tokenizer.word_index # This returns a dictionary, where key->value = word->index. 0 is reserved for padding

NameError: name 'Tokenizer' is not defined

In [12]:
## Convert tokens of text corpus into a sequence of integers based on the word_index dictionary (embedding)
train_sequences = tokenizer.texts_to_sequences(X_train)
#print (X_train[0])
#print (train_sequences[0])
#print (word_index['tv'])

NameError: name 'tokenizer' is not defined

#### Padding

Cuando los textos no tienen el mismo tamaño, hemos de usar *padding*. Esta técnica permite que diferentes piezas de texto acaben con un vector del mismo tamaño, asignando ceros hasta que todas las piezas tengan el mismo tamaño. Esto es necesario para utilizar los textos como inputs para nuestra red neuronal. 

El padding tiene un tamaño máximo que puede ser el del texto más largo, o puede estar fijado por el usuario. En este segundo caso, se pierde información pero por otro lado puede hacer el algoritmo mucho más rápido, en el caso de que tengamos un texto muy muy largo junto a textos mucho más cortos, pues para los textos cortos el vector se vería lleno de ceros sin información útil, ralentizando el proceso. En ningún caso tendremos una longitud de padding mayor que la longitud de los vectores de las *hidden layers* de la red neuronal.

In [13]:
## Create padded training data
train_padded = pad_sequences(train_sequences,
                             maxlen=max_length,
                             padding=padding_type,
                             truncating=trunc_type
                            )

print (len(train_sequences[0]))
print (len(train_padded[0]))

NameError: name 'pad_sequences' is not defined

In [14]:
## Do the same for testing with the trained tokenizer
validation_sequences = tokenizer.texts_to_sequences(X_test)
validation_padded = pad_sequences(validation_sequences,
                                  maxlen=max_length,
                                  padding=padding_type,
                                  truncating=trunc_type
                                 )


NameError: name 'tokenizer' is not defined

In [15]:
##Codify labels
label_tokenizer = Tokenizer()
label_tokenizer.fit_on_texts(y_train)

#Since we have 5 categories, this will be transformed into integer numbers from 1 to 5
train_label_sequences = np.array(label_tokenizer.texts_to_sequences(y_train))
test_label_sequences = np.array(label_tokenizer.texts_to_sequences(y_test))

NameError: name 'Tokenizer' is not defined

In [16]:
## Decoded article (transform from list of numbers back to text)
def decode_article(text):
    reverse_word_index = dict([(value,key) for (key,value) in word_index.items()])
    return ' '.join([reverse_word_index.get(i,'?') for i in text])
print (X_train[10])
print (decode_article(train_padded[10]))

NameError: name 'X_train' is not defined

In [17]:
np.version.version

NameError: name 'np' is not defined

In [18]:
## Build the neural network

model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(embedding_dim)),
    tf.keras.layers.Dense(embedding_dim, activation='relu'),
    tf.keras.layers.Dense(6, activation='softmax')
])
model.summary()

NameError: name 'tf' is not defined